<a href="https://colab.research.google.com/github/garrettfarley9/STL_neighborhood_trees/blob/main/stl_tree_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Money Tree Map
####Using QGIS and Saint Louis Open Data to map tree features against City Block median housing prices

###

##Importing Data

In [1]:
!pip install geopandas
import geopandas as gp

In [5]:
tdf = gp.read_file('/CITY_TREES.dbf')

In [6]:
print(tdf.columns)
print(tdf.shape)

Index(['ON_', 'STREET_NUM', 'STREET', 'COMMON', 'DBH', 'STEMS', 'CONDITION',
       'STORM_DAMA', 'HARDSCAPE_', 'OVERHEAD_U', 'XCOORD', 'YCOORD', 'GUID',
       'ADDRESS', 'ELIGIBLE', 'REPLANT', 'TREATED', 'LOCATION_T', 'LOCATION',
       'geometry'],
      dtype='object')
(118688, 20)


###Nan values

In [13]:

c = tdf[tdf['LOCATION_T'].isna()]
c.head()

,ON_,STREET_NUM,STREET,COMMON,DBH,STEMS,CONDITION,STORM_DAMA,HARDSCAPE_,OVERHEAD_U,XCOORD,YCOORD,GUID,ADDRESS,ELIGIBLE,REPLANT,TREATED,LOCATION_T,LOCATION,geometry
118624,NaN,NaN,NaN,NaN,0,0,N/A,NaN,NaN,NaN,0.0,0.0,D2A6599E-C498-49C8-8A6F-E2C423E53254,3937 RUSSELL AVE,Yes,Blackgum,NaN,NaN,3937 RUSSELL AVE,None
118625,NaN,NaN,NaN,NaN,0,0,N/A,NaN,NaN,NaN,0.0,0.0,87E679ED-3E62-4B23-9C89-CA6C519114A6,7126 MARDEL AVE,Yes,Tuliptree,NaN,NaN,7126 MARDEL AVE,None
118626,NaN,NaN,NaN,NaN,0,0,N/A,NaN,NaN,NaN,0.0,0.0,F7FAFADC-12BC-4CE2-88BB-BE877F8116BB,5716 EICHELBERGER AVE,Yes,"Zelkova, 'Green Vase'",NaN,NaN,5716 EICHELBERGER AVE,None
118627,NaN,NaN,NaN,NaN,0,0,N/A,NaN,NaN,NaN,0.0,0.0,4869CCE0-0C37-4629-B7E1-F9598FBCF730,1022 LOUISVILLE AVE,Yes,"Redbud, Eastern",NaN,NaN,1022 LOUISVILLE AVE,None
118631,NaN,NaN,NaN,NaN,0,0,N/A,NaN,NaN,NaN,0.0,0.0,9890B2E1-B101-4979-8CE1-E2A097E882FB,5517 ITASKA ST,Yes,"Redbud, Eastern",NaN,NaN,5517 ITASKA ST,None


##Cleaning Tree Data

In [6]:
tdf['COMMON'] = tdf['COMMON'].str.capitalize()
tdf.head()

,ON_,STREET_NUM,STREET,COMMON,DBH,STEMS,CONDITION,STORM_DAMA,HARDSCAPE_,OVERHEAD_U,XCOORD,YCOORD,GUID,ADDRESS,ELIGIBLE,REPLANT,TREATED,LOCATION_T,LOCATION,geometry
0,OAK HILL AVE,4067,UTAH ST,"Maple, red",5,1,Fair,No,No,No,889983.173256,1.006871e+06,{000025D7-9880-4FEA-9418-289F5E506BE8},4067 UTAH ST,Yes,NaN,FALSE,Easement Tree,4067 UTAH ST,None
1,SWAN AVE,4427,SWAN AVE,"Maple, silver (restricted use)",18,1,Poor,No,No,No,568477.616427,1.016695e+06,{00004E14-7BFA-4BFA-AE16-B97C20D6C373},4427 SWAN AVE,Yes,NaN,FALSE,Easement Tree,4427 SWAN AVE,None
2,ENRIGHT AVE,5501,ENRIGHT AVE,Vacant,0,0,N/A,No,No,No,882794.969857,1.027355e+06,{00005BB3-56A7-4DE0-8830-2BD5FF695DDE},5501 ENRIGHT AVE,Yes,NaN,FALSE,Easement Tree,5501 ENRIGHT AVE,None
3,BLOW ST,3655,BLOW ST,Vacant,0,0,N/A,No,No,No,887255.794446,9.921068e+05,{000085FC-BD37-4D9D-BBF6-0E3287270D68},3655 BLOW ST,Yes,"Redbud, Eastern",FALSE,Easement Tree,3655 BLOW ST,None
4,COLEMAN AVE,1821,COLEMAN AVE,Vacant,0,0,N/A,No,No,No,899440.025369,1.025467e+06,{0000A65A-A225-4A9C-9FBA-213CA68E6D5F},1821 COLEMAN AVE,Yes,NaN,FALSE,Easement Tree,1821 COLEMAN AVE,None


###Maple tree cleaning

In [19]:
maple = tdf[tdf['COMMON'].str.contains('[Ma]ple', na = False)]
maple = maple[maple['COMMON', 'DBH']]
# .str.capitalize()

# maple.unique()

KeyError: ignored

In [ ]:
import seaborn as sns


###Ash tree cleaning

In [8]:
ash = tdf[tdf['COMMON'].str.contains('[Aa]sh', na = False)]
ash = ash['COMMON'].str.capitalize()
ash.unique()
# def spp(x, species = ''):
#   species =
#   tdf[tdf['COMMON'].str.contains(species, na = False)]

array(['Ash, green', 'Ash, white', 'Ash, pop', 'Mountainash, american',
       'Ash, blue', 'Ash green', 'Ash, european', 'Hawthorn, washington',
       'Ash, spp.', 'Green ash', 'Ash, narrow-leafed', 'Ash, black',
       'Ash'], dtype=object)

###General tree cleaning
Here we are going to apply the same sort of species cleaning for the other genera of trees.

In [9]:
tdf['COMMON'].unique()

array(['Maple, red', 'Maple, silver (restricted use)', 'Vacant',
       'Sweetgum (undesirable)', 'Maple, silver', 'Dogwood, flowering',
       'Maple', 'Oak, pin', 'Oak, english', 'Linden, littleleaf',
       'Pear, callery', 'Stump', 'Maple, freeman', 'Baldcypress, common',
       'Redbud, eastern', 'Sycamore american (restricted use)',
       'Ash, green', 'Ash, white', "Redbud, white 'whitebud'",
       'Maple, sugar', 'Hornbeam, european', 'Crabapple, flowering',
       'Blackgum', 'Hawthorn', 'Serviceberry, spp.', 'Sycamore, american',
       'Elm, hybrid', 'Planetree, london', 'Oak, chinkapin',
       'Cherry/plum, spp.', 'Honeylocust', 'Tuliptree',
       "Serviceberry, apple 'autumn brilliance'",
       'Honeylocust, thornless', 'Ginkgo', "Oak, english 'fastigiata'",
       "Linden, american 'redmond'", 'Oak, pin (restricted use)',
       'Zelkova, japanese', 'Maple, amur', 'Oak, red', 'River, birch',
       'Oak, northern red', 'Oak, sawtooth', 'Oak, swamp white',
       'Map

##Cleaning Housing Data

In [ ]:
import pandas as pd



In [ ]:
bdf = pd.read_csv('/content/PrclSale.csv')
bdf['SaleDate'] = pd.to_datetime(bdf['SaleDate'])

bdf = bdf.loc[bdf['SaleDate'] >= '2010-01-01']
# bdf = bdf.dropna()
bdf = bdf.loc[bdf['SalePrice'] >= 10000]
bdf['block_median']= bdf.groupby('CityBlock')['SalePrice'].transform('median')
bdf = bdf.drop(labels = ['OwnerCode', 'SaleNum', 'AsrParcelId', 'LRMSSaleDate'], axis = 1)

bdf['block_num'] = (bdf['CityBlock'] +10000) * 100
bdf['block_num']= bdf['block_num'].astype(int)
#bdf.tail(50)
bdf.to_csv('median_block')

##Exploratory Data Analysis